# Keet Code Challenge Solution: Jessica Economou

The following document outlines how I solved this coding challenge. Please feel free to reach out if you have any questions! Instead of creating a separate readme file, I thought it would be easier for you to sift through an all-encompassing Jupyter Notebook to review my work.

Thanks again for this opportunity! I had fun with it.

## Environment Setup
I used SQLite to complete this challenge. To interface with this, I used a free SQL client called SQLiteStudio to create a brand new database.

Once the database was defined, I created a new table for the CSV to be imported into by executing the following SQL:

`CREATE TABLE users ( 
    id TEXT
    ,first_name TEXT
    ,last_name TEXT
    ,age INTEGER
    ,gender TEXT
    ,visit_date TEXT
);`

*NOTE: SQLite does not have DATE types for creating schemas, which is why `visit_date` is defined as a TEXT field.*

Next, I read in the CSV using the import tool (ensuring that this import ignored headers) into the `users` table defined above.

I then created a schema for the Daily User counts, to be populated with data in Python (covered below):

`CREATE TABLE daily_user_counts (
    year INTEGER
    ,month INTEGER
    ,day INTEGER
    ,observed TEXT
    ,count REAL
);`

## Read Users Table into Pandas Dataframe

This was a simple task using the SQL Alchemy library to connect to the `users` database, and create a users dataframe using the code below. 

In [171]:
import pandas as pd  
import numpy as np
from sqlalchemy import create_engine 
import datetime
  
conn = create_engine('sqlite:///users.db').connect() 
df = pd.read_sql_table('users', conn)

#convert date to datetime, so that it's easier to parse through
df['visit_date'] = pd.to_datetime(df['visit_date'])
print(df)

               id first_name last_name  age gender visit_date
0   16351203 2040      Leroy    Dillon   55      M 2020-09-07
1   16980104 6773   Macaulay   Griffin   14      M 2020-09-07
2   16230625 0982    Phillip   Chapman   45      M 2020-09-07
3   16180206 2123    Phillip     Moses   39      M 2020-09-07
4   16980621 8401    Tatyana   Goodman   40      F 2020-09-07
..            ...        ...       ...  ...    ...        ...
93  16640909 9774     Kadeem   Lindsay   65      M 2020-09-16
94  16891103 0453   Jermaine      Love   56      M 2020-09-16
95  16510720 9487      Gemma     Baird    6      F 2020-09-16
96  16430629 1396      Brent      Barr   67      M 2020-09-16
97  16340902 2765     Sylvia   Mercado   86      F 2020-09-16

[98 rows x 6 columns]


## Count the number of users by day
After printing out the results of this, I noticed two dates were missing in this dataset. I forced records for these two missing dates by indexing them, so that this can be displayed to include the missing dates (as their counts were in reality 0)

In [245]:
#group by visit date, and take the first two columns (to not get counts replicated across columns)!
date_counts = df.groupby(['visit_date']).count().iloc[:, 0:1]

#force date records
index = pd.date_range('09-07-2020', '09-16-2020')
date_counts.index = pd.DatetimeIndex(date_counts.index)
date_counts = date_counts.reindex(index, fill_value=0)
date_counts.reset_index()

print(date_counts)

            id
2020-09-07   6
2020-09-08   2
2020-09-09  15
2020-09-10  20
2020-09-11  14
2020-09-12   0
2020-09-13   0
2020-09-14  10
2020-09-15  12
2020-09-16  19


## Calculating EXPECTED count, one day in the future
Calculates the number of users expected to signup 1 day into the future. To get one day in the future, this is just a matter of incrementing the max date by 1, and taking an average (without rounding) of the rest of the dataframe to populate its value.

In [227]:
#for getting subsequent date
expected_date = df['visit_date'].max() 
expected_date += datetime.timedelta(days=1)

#for calculating the expected value. I did not round, so this count is the TRUE average.
avg_count = date_counts['id'].mean()

print(expected_date)
print(avg_count)

2020-09-17 00:00:00
9.8


## Bringing it all together
Now I needed to bring this all together in the format requested into the `daily_user_counts` table. I used the following code to do this:

In [235]:
# Columns needed in dataframe: year, month, day, observed, count (for existing counts)
duc = pd.DataFrame()
duc['year'] = date_counts.index.year
duc['month'] = date_counts.index.month
duc['day'] = date_counts.index.day
duc['observed'] = date_counts.index
duc['count'] = date_counts.values

# Append record for "expected" value
expected_data = {
                'year': expected_date.year, 
               'month': expected_date.month, 
               'day': expected_date.day, 
               'observed': expected_date, 
               'count': avg_count
                }
expected_df = pd.DataFrame(expected_data, index=[0]) 

daily_user_counts = duc.append(expected_data, ignore_index=True)
print(daily_user_counts)

    year  month  day   observed  count
0   2020      9    7 2020-09-07    6.0
1   2020      9    8 2020-09-08    2.0
2   2020      9    9 2020-09-09   15.0
3   2020      9   10 2020-09-10   20.0
4   2020      9   11 2020-09-11   14.0
5   2020      9   12 2020-09-12    0.0
6   2020      9   13 2020-09-13    0.0
7   2020      9   14 2020-09-14   10.0
8   2020      9   15 2020-09-15   12.0
9   2020      9   16 2020-09-16   19.0
10  2020      9   17 2020-09-17    9.8


## Writing results to SQL 
Using the SQL Alchemy library again, I wrote the results back to the existing table I created earlier (without a column for the index). I double checked that results were showing up as expected in SQL, and everything looked great!

In [244]:
daily_user_counts.to_sql('daily_user_counts', con = conn, index = False, if_exists = 'replace')

## How I'd improve my solution in the future
1. For technical challenge purposes, this was a great setup and I had a good time with it. If this was a real life challenge I was solving though, using Python for this minimal computation work felt excessive. This could have all been done within the database using SQL, so that data wouldn't have to be duplicated again in Python dataframes.
2. I probably wouldn't choose SQLite, as it doesn't have support for storing DATE objects. This was the most convenient one for me though, as it's free and I already had it installed on my computer.
3. Since this was more of a quick challenge with a very small and static table, I didn't specify whether columns in these tables were null/not null, or bother with creating indexes in the database. If this was a true database setup where we could expect to read in data every day, I would be sure to focus on getting these right!